### Loading and Cleaning Data

1. **Loading and Preprocessing Orbis Data:**
   - **Rename Columns:** The Orbis dataset columns are renamed according to `orbis_column_map`.
   - **Clean Employee Data:** Employee numbers are converted to numeric format using `clean_employee_data`.
   - **Preprocess Company Names:** Company names are cleaned by removing special characters, excess whitespace, and converting to lowercase with `preprocess_company_name`.
   - **Filter by Company Size:** Companies with fewer than 30 employees are filtered out.
   - **Remove Duplicates:** Duplicate company names are dropped, retaining only the first instance.

2. **Loading and Preprocessing Kununu Data:**
   - **Sort by Reviews:** The Kununu dataset is sorted by `firm_name` and `total_reviews_num`, with companies having the most reviews prioritized.
   - **Remove Duplicates:** Duplicates are removed based on `firm_name`, keeping the record with the highest review count.
   - **Rename Columns:** Columns are renamed using `kununu_column_map`.
   - **Filter by Reviews:** Companies with fewer than 10 reviews are excluded.
   - **Preprocess Company Names:** Company names are cleaned similarly to the Orbis dataset.

3. **Sequence of Matching Steps:**
   - **Exact Match:** An initial attempt to find exact matches between Orbis and Kununu company names.
   - **Standardize Abbreviations:** Abbreviations in company names are standardized.
   - **Remove Umlauts:** Umlauts in German company names are converted to their English equivalents.
   - **Remove Suffixes:** Common suffixes like "GmbH" are removed.
   - **Remove Common Words:** Frequently appearing words are removed from company names.


In [3]:
from rapidfuzz import fuzz, process
from mappings import *
from utils import *
import pandas as pd
import numpy as np                               

# Load and preprocess Orbis data
orbis = (
    pd.read_csv("data/full_german_sample.csv", low_memory=False)
    .rename(columns=orbis_column_map)                                               # rename columns
    .assign(
        employees=lambda df: df["employees"].apply(clean_employee_data),            # convert employee data to numeric
        orbis_name=lambda df: df["orbis_name"].apply(preprocess_company_name)       # remove special characters and whitespace and make lowercase
    )
    .query("employees > 30")[orbis_column_map.values()]                             # remove companies with less than 30 employees
    .drop_duplicates(subset="orbis_name", keep="first")                             # drop duplicates
)

# Load and preprocess Kununu data
kununu = (
    pd.read_csv("data/kununu.csv")
    .sort_values(by=["firm_name", "total_reviews_num"], ascending=[True, False])
    .drop_duplicates(subset="firm_name", keep="first")                              # drop duplicates -- keep entry with most reviews
    .rename(columns=kununu_column_map)
    .query("total_reviews_num > 10")[["kununu_name", "kununu_id"]]                  # remove companies with less than 10 reviews
    .assign(kununu_name=lambda df: df["kununu_name"].apply(preprocess_company_name))
)

# get the list of words that appear more commonly than 99% of the words in company names
common_words = get_most_common_words(list(orbis["orbis_name"]) + list(kununu["kununu_name"]))

# preserving original names before modifications
orbis.loc[:, "original_orbis_name"] = orbis["orbis_name"]
kununu.loc[:, "original_kununu_name"] = kununu["kununu_name"]

# define the sequence of matching steps
steps = [
    (lambda x: x, "exact_match"),
    (standardize_abbreviations, "standardize_abbreviations"),
    (umlauts_to_english, "remove_umlauts"),
    (remove_suffixes, "remove_suffixes"),
    (lambda x: remove_words_from_name(x, common_words), "remove_common_words"),
]
orbis, kununu, all_matches = match_firms_in_sequence(orbis, kununu, steps)

Number of matches after applying exact_match: 13791
Number of matches after applying standardize_abbreviations: 431
Number of matches after applying remove_umlauts: 32
Number of matches after applying remove_suffixes: 1258
Number of matches after applying remove_common_words: 3695
Matched 49.87% of Kununu firms.


### Matching Remaining Firms Using Fuzzy Merge

Calculating and filtering fuzzy similarity scores between company names in the Orbis and Kununu datasets, then preparing DataFrame with the best matches for further verification:

1. **Calculate Similarity Scores:**
   - **Compute Fuzzy Similarity:** Calculate similarity scores between Orbis and Kununu company names using the `fuzz.QRatio` scorer.
   - **Filter by Threshold:** Only matches with a similarity score of 90 or higher are retained.

2. **Create DataFrame for Best Matches:**
   - **Create DataFrame:** Include these columns: `Original Orbis Name`, `Original Kununu Name`, `Similarity Score`, `Orbis Name (after preprocessing)`, `Kununu Name (after preprocessing)`, `Orbis ID`, `Kununu ID`, `NACE Code`, `Employees`, and `City`.
   - **Sort Matches:** The DataFrame is sorted by similarity score (in descending order) and then by Kununu ID (in ascending order).

3. **Translate Kununu Names:**
   - **Translate to English:** The `Original Kununu Name` is translated to English using the `translate_text` function to assist in verifying matches.


In [15]:
similarity_scores_matrix = process.cdist(                                         # calculate similarity scores between Orbis and Kununu names
    orbis["orbis_name"], kununu["kununu_name"], scorer=fuzz.QRatio
)
filtered_indices = np.where(similarity_scores_matrix >= 90)                       # remove matches with similarity score less than 90
best_fuzzy_matches = pd.DataFrame(                                                # create a DataFrame with the best fuzzy matches to verify
    {
        "Original Orbis Name": orbis["original_orbis_name"].iloc[filtered_indices[0]].values,
        "Original Kununu Name": kununu["original_kununu_name"].iloc[filtered_indices[1]].values,
        "Similarity Score": similarity_scores_matrix[filtered_indices],
        "Orbis Name (after preprocessing)": orbis["orbis_name"].iloc[filtered_indices[0]].values,
        "Kununu Name (after preprocessing)": kununu["kununu_name"].iloc[filtered_indices[1]].values,
        "Orbis ID": orbis["id"].iloc[filtered_indices[0]].values,
        "Kununu ID": kununu["kununu_id"].iloc[filtered_indices[1]].values,
        "NACE Code": orbis["nace_code"].iloc[filtered_indices[0]].values,
        "Employees": orbis["employees"].iloc[filtered_indices[0]].values,
        "City": orbis["city"].iloc[filtered_indices[0]].values,
    }
).sort_values(["Similarity Score", "Kununu ID"], ascending=[False, True])         # sort by similarity score and Kununu ID
best_fuzzy_matches["Kununu Name Translated"] = best_fuzzy_matches[
    "Original Kununu Name"
].apply(translate_text)                                                           # translate Kununu names to English to help verify matches
best_fuzzy_matches.to_csv("data/fuzzy_matches_to_verify.csv", index=False)        # save the best fuzzy matches to a CSV file

Shape: (9457, 10)
